In [103]:
import csv
from qiskit import transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_distribution
from qiskit_aer import AerSimulator
import qiskit.quantum_info as qi
import numpy as np
from quantum_check import construct_circuit, check_consistency
import matplotlib.pyplot as plt
import pandas as pd

In [88]:
#Load in data from data.csv
test_data = []

with open("data/data.csv", mode="r", newline="") as file: 
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        n = int(row[0])
        system_type = row[1]
        F1, F2, F3 = row[2], row[3], row[4]
        test_data.append((n, system_type, F1, F2, F3))

In [ ]:
def theoretical_quantum_test():
    exp = 0
    terms = []
    func = 0
    var = 0
    label = ''

    for data in test_data:
        exp += 1
        var = data[0]
        label = data[1]
        for i in range(len(data) - 2):
            terms.append(data[i+2])
        func = len(terms)
        
        circ = construct_circuit(terms, var, func)
        # circ.draw("mpl");
        # plt.savefig(f"theoretical_figures/circuit_{exp}_{terms}")
        # plt.close()

        simulator = AerSimulator(method='matrix_product_state')
        circ = transpile(circ, simulator)

        shots = [64, 128, 256]
        
        a_probs = []
        a_probs.append(exp)
        a_probs.append(var)
        
        for shot in shots:
            result = simulator.run(circ, shots=shot).result()
            probabilities = {k:v/shot for k,v in result.get_counts().items()}

            p0011 = probabilities.get('0011', 0)
            p1100 = probabilities.get('1100', 0)
            
            a_probs.append(probabilities.get('0000', 0))
            a_probs.append(probabilities.get('0011', 0))
            a_probs.append(probabilities.get('1100', 0))
            a_probs.append(probabilities.get('1111', 0))
        
        with open('data/theoretical_probabilities.csv', mode='a', newline='') as file: #64x000,64x0011,64x1100,64x1111,128x000,128x0011,128x1100,256x1111,256x000,256x0011,256x1100,256x1111
            writer = csv.writer(file)
            writer.writerow(a_probs)
            
        #print("\n");
        
        terms = []


In [118]:
for i in range(100):
    theoretical_quantum_test();

In [119]:
#Data analysis
df = pd.read_csv("data/theoretical_probabilities.csv")
df_without_var = df.drop(columns=['var'])
df_sorted = df.sort_values(by='experiment')
df_avg = df_sorted.groupby('experiment').mean()
df_avg.reset_index(inplace=True)
df_avg.to_csv('data/theoretical_mean_results.csv', index=False)

In [ ]:
df = pd.read_csv('data/theoretical_mean_results.csv')

labels = ['0000', '0011', '1100', '1111']

for i, row in df.iterrows():
    var_value = row['var']
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    
    for j in range(3):
        start_idx = 2 + j * 4
        axs[j].bar(labels, row[start_idx:start_idx + 4])
        axs[j].set_title(f"Experiment {i+1} - Graph {j+1}: {2**(j+6)} Shots")
        axs[j].set_xlabel('Labels')
        axs[j].set_ylabel('Quasi-Probabilities')

    plt.tight_layout()
    fig.savefig(f"figures/theoretical_figures/experiment_{i+1}_var{var_value}.png")
    plt.close(fig)